### Video Frame to EEG Sample alignment method

During initial investigations, we noticed that calculating the offset between the starting point of the EEG and the first TTL onset, and the start of the video recording and the frist LED onset is not enough. Somehow, there's either lag in the EEG recording, or the theoretical 30 FPS of the video recording is not reached in practice. Therefore we need a method that repairs this systematically increasing lag between the two in order to align them properly.

In [1]:
import os
import cv2
import json
import pickle
import ndx_events
import numpy as np
from pynwb import NWBHDF5IO

In [3]:
with open('../settings.json', "r") as f:
    settings = json.load(f)
    
epoch_folder = settings['epochs_folder']
plot_folder = settings['plots_folder']
nwb_folder = settings['nwb_files_folder']

Let's create some handy converting functions

In [40]:
def sample_to_frame(eeg_tp_in_secs, fps, offset):
    """
    Function that calculates the time-point of the video (in frames) given
     the time-point (in seconds) in the EEG.
    """
    video_tp_secs = eeg_tp_in_secs - offset  # subtract the offset so we have the video tp in secs

    return video_tp_secs * fps  # go to frames

In [42]:
def frame_to_sample(frame_number, fps, offset):
    """
    Function that calculates the time-point of the EEG (in secs) given
     the video time-point in frames.
    """
    tp_secs_video = frame_number / fps  # time-point on video in seconds
    secs_eeg = tp_secs_video + offset
    
    return secs_eeg

### Loading of needed info

For the converting process we need a couple of things:
* The EEG TTL onsets (which are in seconds)
* A full EEG signal (does not matter which channel)
* The sampling frequency of the EEG in the NWB file
* The LED onset time-points (in frames)

We do this only for a test subject that is manually selected.

In [14]:
test_subject = "80630"  # is in batch 1 (see metadata excel sheets)
video_filename = "drd2_batch4_resting-state Camera 1 13-10-2023 09_29_44 1.mp4"

In [15]:
eeg_ttl_onsets_secs = []  # container (to make it accessible through notebook)

for file in os.listdir(nwb_folder):
    if test_subject in file and file.endswith(".nwb"):
        with NWBHDF5IO(f'{nwb_folder}/{file}', "r") as io:  # open it
            nwb = io.read()
            eeg_ttl_onsets_secs = list(nwb.acquisition["TTL_1"].timestamps)
            s_freq = nwb.acquisition['raw_EEG'].rate
            eeg_signal = filtered_eeg = nwb.acquisition['filtered_EEG'].data[:].T[0]

Now that we have the TTL onset timepoints from the EEG of the test_subject, let's load the data that holds the LED ON timepoints from the accompanying video file.

In [16]:
folder_path = "/Users/olledejong/Documents/MSc_Biology/ResearchProject2/rp2_data/resting_state/output/videos"
pickle_path = f"{folder_path}/pickle"

with open(f'{pickle_path}/led_states_all_videos.pickle', "rb") as f:
    led_states = pickle.load(f)

In [17]:
led_states_data = led_states[video_filename]

We only need the frame numbers where the LED switches ON, not all frames where the LED is ON.

In [36]:
led_turns_on_indexes = np.where(np.logical_and(np.diff(led_states_data), led_states_data[1:]))[0] + 1  # get all False to True changes in the array
led_turns_on_indexes

array([   220,    250,    280, 548578, 548608, 548638])

Using the frame number where the LED turns ON for the first time, we can calculate the needed offset.
We calculate this by subtracting the time elapsed between start of **video** and the first LED onset from the time elapsed between start of **EEG** recording and the first TTL onset.

In [23]:
video_fps = 30
first_ttl_onset = eeg_ttl_onsets_secs[0]
first_LED_onset = led_turns_on_indexes[0]

offset_reg_fps = first_ttl_onset - first_LED_onset / video_fps

8.66896666666667 8.663033675800019


#### Try alignment with theoretical FPS
Let's see how the aligning goes when we use the theoretical fps (30) and the offset that is calculated using that value.

In [25]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=video_fps, offset=offset_reg_fps)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}. Delta: {eeg_ttl_onsets_secs[i] - eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 16.0023, calculated: 15.996367009133351. Delta: 0.005932990866650556
Onset 1. Actual EEG TTL onset: 17.0044, calculated: 16.99636700913335. Delta: 0.008032990866649214
Onset 2. Actual EEG TTL onset: 18.0065, calculated: 17.99636700913335. Delta: 0.010132990866647873
Onset 3. Actual EEG TTL onset: 18309.3886, calculated: 18294.596367009133. Delta: 14.792232990865159
Onset 4. Actual EEG TTL onset: 18310.3898, calculated: 18295.596367009133. Delta: 14.793432990867586
Onset 5. Actual EEG TTL onset: 18311.3919, calculated: 18296.596367009133. Delta: 14.795532990865468


Now let's try the other way around

In [29]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=video_fps, s_freq=s_freq, offset=offset_reg_fps)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 220, calculated: -259.62642549504983
Onset 1. Actual frame LED onset: 250, calculated: -259.5987104846194
Onset 2. Actual frame LED onset: 280, calculated: -259.57099547418903
Onset 3. Actual frame LED onset: 548578, calculated: 246.31249488425988
Onset 4. Actual frame LED onset: 548608, calculated: 246.34018500345252
Onset 5. Actual frame LED onset: 548638, calculated: 246.36790001388283


#### Now, let's try with the adjusted FPS

Lets calculate the adjusted fps to account for the delay in either the video or eeg recording.

In [21]:
# Find length of eeg signal between the two pulse combination. For example first and last
eeg_len = eeg_signal[int(s_freq * eeg_ttl_onsets_secs[0]): int(s_freq * eeg_ttl_onsets_secs[-1])].shape[0]

# find length of video frames between the two pulse combination
frame_len = led_turns_on_indexes[-1] - led_turns_on_indexes[0]

adjusted_fps = (frame_len / (eeg_len / s_freq))
adjusted_fps

29.975748294429295

So, the actual framerate is not exactly 30. Now, lets re-calculate the offset with this adjusted fps as well.

In [ ]:
offset_adj_fps = first_ttl_onset - first_LED_onset / adjusted_fps

And let's test the alignment of EEG and Video again

In [30]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=adjusted_fps, offset=offset_adj_fps)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 16.0023, calculated: 16.0023
Onset 1. Actual EEG TTL onset: 17.0044, calculated: 17.00310904420909
Onset 2. Actual EEG TTL onset: 18.0065, calculated: 18.00391808841818
Onset 3. Actual EEG TTL onset: 18309.3886, calculated: 18309.390495480246
Onset 4. Actual EEG TTL onset: 18310.3898, calculated: 18310.391304524455
Onset 5. Actual EEG TTL onset: 18311.3919, calculated: 18311.392113568665


In [39]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=adjusted_fps, offset=offset_adj_fps)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 220, calculated: 220.0
Onset 1. Actual frame LED onset: 250, calculated: 250.03869736584755
Onset 2. Actual frame LED onset: 280, calculated: 280.0773947316951
Onset 3. Actual frame LED onset: 548578, calculated: 548577.9431815612
Onset 4. Actual frame LED onset: 548608, calculated: 548607.9549007537
Onset 5. Actual frame LED onset: 548638, calculated: 548637.9935981195
